# Tokenizer installation

pip install tokenizers

Please see 
- https://github.com/huggingface/tokenizers/blob/master/README.md
- https://github.com/huggingface/tokenizers/tree/master/bindings/python

In [1]:
from tokenizers import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing
import torch
from transformers import BertForMaskedLM 

In [2]:
tokenizer = BertWordPieceTokenizer("bert-base-swedish-uncased-vocab.txt", lowercase=True, strip_accents=False)

In [3]:
example = "Jag är ett barn, och det här är mitt hem. Alltså är det ett barnhem!"

output = tokenizer.encode(example)

In [14]:
print(output.tokens[17])

barn


In [6]:
indexed_tokens = output.ids
masked_index = 17  # 'barn'
indexed_tokens[masked_index] = tokenizer.token_to_id('[MASK]')

In [7]:
indexed_tokens_tensor = torch.tensor([indexed_tokens])
print(indexed_tokens_tensor)

tensor([[ 101, 1112, 1100, 1115, 1255, 1010, 1095, 1102, 1174, 1100, 1352, 1345,
         1012, 1492, 1100, 1102, 1115,  103, 2760,  999,  102]])


In [8]:
model = BertForMaskedLM.from_pretrained('af-ai-center/bert-base-swedish-uncased')
_ = model.eval()

In [9]:
# predict all tokens
with torch.no_grad():
    outputs = model(indexed_tokens_tensor)

predictions = outputs[0]
print(predictions.shape)  # 1 example, 21 tokens, 30522 vocab size

torch.Size([1, 21, 30522])


In [10]:
# show predicted index for masked token
predicted_index = torch.argmax(predictions[0, masked_index])
print(predicted_index)

tensor(1255)


In [11]:
# show top5 predicted index for masked token
predicted_index_top5 = torch.argsort(predictions[0, masked_index], descending=True)[:5]

In [12]:
tokenizer.decode(predicted_index_top5.numpy())

'barn foster barndoms dock dag'

In [13]:
# show top5 predicted masked token
for predicted_index in predicted_index_top5:
    predicted_token = tokenizer.id_to_token(predicted_index)
    text = output.original_str[:output.offsets[masked_index][0]] + predicted_token + output.original_str[output.offsets[masked_index][1]:]
    print(text)

jag är ett barn, och det här är mitt hem. alltså är det ett barnhem!
jag är ett barn, och det här är mitt hem. alltså är det ett fosterhem!
jag är ett barn, och det här är mitt hem. alltså är det ett barndomshem!
jag är ett barn, och det här är mitt hem. alltså är det ett dockhem!
jag är ett barn, och det här är mitt hem. alltså är det ett daghem!
